In [1]:
from mlflow import MlflowClient
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor

client = MlflowClient(tracking_uri="http://127.0.0.1:8081")

In [2]:
# view metadata associated with experiments on the server
all_experiments = client.search_experiments()
all_experiments

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1749644300253, experiment_id='0', last_update_time=1749644300253, lifecycle_stage='active', name='Default', tags={}>]

In [3]:
default_experiment = [
    {"name": experiment.name, "lifecycle_stage": experiment.lifecycle_stage}
    for experiment in all_experiments
    if experiment.name == "Default"
][0]

pprint(default_experiment)

{'lifecycle_stage': 'active', 'name': 'Default'}


In [10]:
# experiment description
experiment_description = ("This is the grocery forecasting project. "
                        "This experiment contains the produce models for apples.")

# searchable tags that define charcateristics of the Runs that will be in this experiment
experiment_tags = {
    'project_name':"grocery-forecasting",
    'store_dept': 'produce',
    'team': 'stores-ml',
    'project_quarter': 'Q3-2023',
    'mlflow.note.content': experiment_description,
}

# create experiment
produce_apples_experiment = client.create_experiment(
    name="Apple_Models", tags = experiment_tags
)

In [11]:
apples_experiment = client.search_experiments(
    filter_string="tags.`project_name`='grocery-forecasting'"
)

vars(apples_experiment[0])

{'_experiment_id': '793511272458510520',
 '_name': 'Apple_Models',
 '_artifact_location': 'mlflow-artifacts:/793511272458510520',
 '_lifecycle_stage': 'active',
 '_tags': {'project_name': 'grocery-forecasting',
  'project_quarter': 'Q3-2023',
  'mlflow.note.content': 'This is the grocery forecasting project.This experiment contains the produce models for apples.',
  'store_dept': 'produce',
  'team': 'stores-ml'},
 '_creation_time': 1749645547132,
 '_last_update_time': 1749645547132}

In [13]:
apples_experiment[0].tags['team']

'stores-ml'

In [14]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [15]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


def generate_apple_sales_data_with_promo_adjustment(
    base_demand: int = 1000, n_rows: int = 5000
):
    """
    Generates a synthetic dataset for predicting apple sales demand with seasonality
    and inflation.

    This function creates a pandas DataFrame with features relevant to apple sales.
    The features include date, average_temperature, rainfall, weekend flag, holiday flag,
    promotional flag, price_per_kg, and the previous day's demand. The target variable,
    'demand', is generated based on a combination of these features with some added noise.

    Args:
        base_demand (int, optional): Base demand for apples. Defaults to 1000.
        n_rows (int, optional): Number of rows (days) of data to generate. Defaults to 5000.

    Returns:
        pd.DataFrame: DataFrame with features and target variable for apple sales prediction.

    Example:
        >>> df = generate_apple_sales_data_with_seasonality(base_demand=1200, n_rows=6000)
        >>> df.head()
    """

    # Set seed for reproducibility
    np.random.seed(9999)

    # Create date range
    dates = [datetime.now() - timedelta(days=i) for i in range(n_rows)]
    dates.reverse()

    # Generate features
    df = pd.DataFrame(
        {
            "date": dates,
            "average_temperature": np.random.uniform(10, 35, n_rows),
            "rainfall": np.random.exponential(5, n_rows),
            "weekend": [(date.weekday() >= 5) * 1 for date in dates],
            "holiday": np.random.choice([0, 1], n_rows, p=[0.97, 0.03]),
            "price_per_kg": np.random.uniform(0.5, 3, n_rows),
            "month": [date.month for date in dates],
        }
    )

    # Introduce inflation over time (years)
    df["inflation_multiplier"] = (
        1 + (df["date"].dt.year - df["date"].dt.year.min()) * 0.03
    )

    # Incorporate seasonality due to apple harvests
    df["harvest_effect"] = np.sin(2 * np.pi * (df["month"] - 3) / 12) + np.sin(
        2 * np.pi * (df["month"] - 9) / 12
    )

    # Modify the price_per_kg based on harvest effect
    df["price_per_kg"] = df["price_per_kg"] - df["harvest_effect"] * 0.5

    # Adjust promo periods to coincide with periods lagging peak harvest by 1 month
    peak_months = [4, 10]  # months following the peak availability
    df["promo"] = np.where(
        df["month"].isin(peak_months),
        1,
        np.random.choice([0, 1], n_rows, p=[0.85, 0.15]),
    )

    # Generate target variable based on features
    base_price_effect = -df["price_per_kg"] * 50
    seasonality_effect = df["harvest_effect"] * 50
    promo_effect = df["promo"] * 200

    df["demand"] = (
        base_demand
        + base_price_effect
        + seasonality_effect
        + promo_effect
        + df["weekend"] * 300
        + np.random.normal(0, 50, n_rows)
    ) * df[
        "inflation_multiplier"
    ]  # adding random noise

    # Add previous day's demand
    df["previous_days_demand"] = df["demand"].shift(1)
    df["previous_days_demand"].fillna(
        method="bfill", inplace=True
    )  # fill the first row

    # Drop temporary columns
    df.drop(columns=["inflation_multiplier", "harvest_effect", "month"], inplace=True)

    return df

In [16]:
data = generate_apple_sales_data_with_promo_adjustment(base_demand=1_000, n_rows=1_000)

data[-20:]

,date,average_temperature,rainfall,weekend,holiday,price_per_kg,promo,demand,previous_days_demand
980,2025-05-23 15:03:16.718834,34.130183,1.454065,0,0,1.449177,0,999.306290,1029.418398
981,2025-05-24 15:03:16.718833,32.353643,9.462859,1,0,2.856503,0,1169.129427,999.306290
982,2025-05-25 15:03:16.718832,18.816833,0.391470,1,0,1.326429,0,1317.616709,1169.129427
983,2025-05-26 15:03:16.718831,34.533012,2.120477,0,0,0.970131,0,1068.802075,1317.616709
984,2025-05-27 15:03:16.718830,23.057202,2.365705,0,0,1.049931,0,1019.486305,1068.802075
985,2025-05-28 15:03:16.718829,34.810165,3.089005,0,0,2.035149,0,1002.564672,1019.486305
986,2025-05-29 15:03:16.718828,29.208905,3.673292,0,0,2.518098,0,1086.143402,1002.564672
987,2025-05-30 15:03:16.718827,16.428676,4.077782,0,0,1.268979,0,1093.207186,1086.143402
988,2025-05-31 15:03:16.718826,32.067512,2.734454,1,0,0.762317,0,1396.939894,1093.207186
989,2025-06-01 15:03:16.718825,31.938203,13.883486,1,0,1.153301,0,1321.409540,1396.939894
